# 🟠 Lesson 10: Inpainting Masks

Inpainting works just like Image-to-Image, but we only noise/denoise **specific pixels** defined by a black-and-white mask.

### Challenge: The VAE Problem
Remember the VAE shrinks images 8x? (512 -> 64).
This means if you mask 1 single pixel, the VAE cannot represent it perfectly. It operates on 8x8 patches.
This is why inpainting masks sometimes feel "blocky" at the edges.

In [ ]:
# 1. Setup
import notebook_utils
project_root, device, dtype = notebook_utils.setup_notebook()

from PIL import Image, ImageDraw
from core.inpainter import inpainter
import requests
from io import BytesIO

## 1. Prepare Image and Mask
We will manually create a mask using code.

In [ ]:
# Load dog image
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/inpainting/overture-creations-5sI6fQgYIuo.png"
image = Image.open(BytesIO(requests.get(url).content)).resize((512, 512))

# Create mask (White box in the middle)
mask = Image.new("L", (512, 512), 0) # Black background
draw = ImageDraw.Draw(mask)
draw.rectangle((128, 128, 384, 384), fill=255) # White square

notebook_utils.show_image(image, title="Original")
notebook_utils.show_image(mask, title="Mask (White = Edit)")

## 2. Inpaint

We use the project's `Inpainter` class.

In [ ]:
prompt = "a cute robot sitting on a bench"

result, _ = inpainter.inpaint(
    image=image,
    mask=mask,
    prompt=prompt,
    num_steps=30,
    blur_mask=5
)

notebook_utils.show_image(result, title="Result")

## 3. Mask Blurring

Because of the VAE 8x8 block issue, we usually **blur the mask** (gaussian blur) before sending it to the model. This creates a smoother transition between the edited area and the original pixels.